Now that we've seen the story that the data tells, it's time to build the model. First we import the libraries and the data file. 

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale

print("Loaded")

Loaded


In [2]:
df = pd.read_csv (r"C:\Users\Laura-Black\Documents\PhD\Data Scientist\Springboard\Capstone Projects\Drug Abuse ED Visits\Output files\Capstone1DataWrangling.csv", 
                     index_col=0, low_memory=False)

print(df.head())
print(df.info())

              METRO  STRATA  PSU  REPLICATE   CASEWGT  PSUFRAME AGECAT  \
CASEID                                                                   
1          New York      25  108          2  0.942635         3  18-20   
2          New York      29  129          2  5.992011         9   > 65   
3       Date County       7   25          1  4.723172         6   > 65   
4           Phoenix       8   29          2  4.080147         6  06-11   
5            Boston      22   94          2  5.177709        10  25-29   

           SEX      RACE  YEAR  ...       DRUGID_22 ROUTE_22  TOXTEST_22  \
CASEID                          ...                                        
1         Male     Black  2011  ...  Not Applicable      NaN         NaN   
2         Male  Hispanic  2011  ...  Not Applicable      NaN         NaN   
3       Female     Black  2011  ...  Not Applicable      NaN         NaN   
4         Male  Hispanic  2011  ...  Not Applicable      NaN         NaN   
5         Male  Hispanic 

Now, we will create a subset of the data to begin testing.

In [6]:
df_sub_col = df[['METRO','STRATA','AGECAT','SEX','RACE','QUARTER','DAYPART','DRUGID_1','ROUTE_1','TOXTEST_1',
                'DRUGID_2','ROUTE_2','TOXTEST_2','DRUGID_3','ROUTE_3','TOXTEST_3','CASETYPE','DISPOSITION','ALCOHOL',
                'NONALCILL','PHARMA','NONMEDPHARMA','ALLABUSE']]

df_sub = df_sub_col.sample(frac = 0.25)
print(df_sub.head())
print()
print()
print(df_sub.info())

            METRO  STRATA AGECAT     SEX     RACE  QUARTER        DAYPART  \
CASEID                                                                      
220162   New York      25  35-44    Male    White        4  6:00-11:59 PM   
157720    Seattle      26  45-54  Female    White        2  12:00-5:59 PM   
158659   New York       3  35-44  Female    Black        3  6:00-11:59 PM   
156670  All Other      31  35-44  Female    Black        2  6:00-11:59 PM   
142659     Denver       2  12-17  Female  Not Doc        1  6:00-11:59 PM   

           DRUGID_1                    ROUTE_1      TOXTEST_1  ...  \
CASEID                                                         ...   
220162       HEROIN                    Not Doc  Not Confirmed  ...   
157720  LAMOTRIGINE                    Not Doc  Not Confirmed  ...   
158659       HEROIN  Inhaled, Sniffed, Snorted  Not Confirmed  ...   
156670      ALCOHOL                    Not Doc  Not Confirmed  ...   
142659   RANITIDINE                    N

Now, we will split the data into X and y.

In [7]:
x_split = df_sub.drop('CASETYPE', axis=1)

X = pd.get_dummies(x_split,drop_first=True)

In [8]:
print(X)

        STRATA  QUARTER  METRO_Boston  METRO_Chicago  METRO_Date County  \
CASEID                                                                    
220162      25        4             0              0                  0   
157720      26        2             0              0                  0   
158659       3        3             0              0                  0   
156670      31        2             0              0                  0   
142659       2        1             0              0                  0   
...        ...      ...           ...            ...                ...   
189880      40        4             0              0                  0   
47342       20        2             0              0                  0   
134714      29        2             0              0                  0   
5750         4        2             0              0                  0   
227439       2        1             0              0                  0   

        METRO_Denver  ME

In [22]:
#create y
y = df_sub['CASETYPE'].values

In [23]:
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42)

In [43]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

34381
22922
34381
22922


In [44]:
# Create the classifier: logreg
logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs', max_iter=10000)

# Fit the classifier to the training data
logreg.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred_test = logreg.predict(X_test)
y_pred_train = logreg.predict(X_train)

# Compute and print the confusion matrix and classification report
print('Test Set')
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
print(logreg.score(X_test,y_test))

print('Train Set')
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print(logreg.score(X_train,y_train))

Test Set
[[ 159  138    0    0    6    0    1    5]
 [  42 8604    0    0    2    0   30   94]
 [   0    0  736    0    0    0    0    0]
 [   0    0    0   34   51    3    0    0]
 [   0    0    0    4 8269  409  144    0]
 [   0    0    0    1  496 1332    0    0]
 [   0   87    3    0  470    0  837   67]
 [   1  241    4    0   61    0   75  516]]
                       precision    recall  f1-score   support

Accidential Ingestion       0.79      0.51      0.62       309
     Adverse Reaction       0.95      0.98      0.96      8772
     Alcohol (Age<21)       0.99      1.00      1.00       736
  Malicious Poisoning       0.87      0.39      0.54        88
                Other       0.88      0.94      0.91      8826
       Overmedication       0.76      0.73      0.75      1829
        Seeking Detox       0.77      0.57      0.66      1464
      Suicide Attempt       0.76      0.57      0.65       898

             accuracy                           0.89     22922
            ma

Next, we will perform a logistic regression analysis on the full data set. We will start by redefining the x and y variables.

In [45]:
x_split = df.drop('CASETYPE', axis=1)

X = pd.get_dummies(x_split,drop_first=True)

In [48]:
y = df['CASETYPE'].values

In [49]:
# Recreate training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42, stratify=y)

In [56]:
# Create the classifier: logreg
logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs', max_iter=1000000000)

# Fit the classifier to the training data
logreg.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred_test = logreg.predict(X_test)
y_pred_train = logreg.predict(X_train)

# Compute and print the confusion matrix and classification report
print('Test Set')
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
print(logreg.score(X_test,y_test))


C:\Users\Laura-Black\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Test Set
[[   80  1152     0     0    25     0     1    43]
 [   21 34716     0     0     8     0    87   407]
 [    0     0  2968     0     0     0     0     0]
 [    0     0     0    68   232    17     0     0]
 [    0     0     0    22 32494  1900   635     0]
 [    0     0     0     4  1762  5493     0     0]
 [    0   455    10     0  2003     0  3212   257]
 [    2  1109    11     0   228     0   310  1953]]
                       precision    recall  f1-score   support

Accidential Ingestion       0.78      0.06      0.11      1301
     Adverse Reaction       0.93      0.99      0.96     35239
     Alcohol (Age<21)       0.99      1.00      1.00      2968
  Malicious Poisoning       0.72      0.21      0.33       317
                Other       0.88      0.93      0.91     35051
       Overmedication       0.74      0.76      0.75      7259
        Seeking Detox       0.76      0.54      0.63      5937
      Suicide Attempt       0.73      0.54      0.62      3613

             

print('Train Set')
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print(logreg.score(X_train,y_train))

In [57]:
# Create the classifier: logreg
logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs', max_iter=1000000000)

# Fit the classifier to the training data
logreg.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred_test = logreg.predict(X_test)
y_pred_train = logreg.predict(X_train)

# Compute and print the confusion matrix and classification report
print('Train Set')
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print(logreg.score(X_train,y_train))

C:\Users\Laura-Black\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Train Set
[[  144  1686     0     0    50     0     2    70]
 [   28 52121     0     0     9     0   118   581]
 [    0     0  4453     0     0     0     0     0]
 [    0     0     0   129   327    19     1     0]
 [    0     0     0    20 48718  2871   968     0]
 [    0     0     0     7  2652  8228     0     0]
 [    0   726    17     0  2987     0  4823   351]
 [    1  1594    27     0   346     0   471  2981]]
                       precision    recall  f1-score   support

Accidential Ingestion       0.83      0.07      0.14      1952
     Adverse Reaction       0.93      0.99      0.96     52857
     Alcohol (Age<21)       0.99      1.00      1.00      4453
  Malicious Poisoning       0.83      0.27      0.41       476
                Other       0.88      0.93      0.90     52577
       Overmedication       0.74      0.76      0.75     10887
        Seeking Detox       0.76      0.54      0.63      8904
      Suicide Attempt       0.75      0.55      0.63      5420

            